In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [4]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [5]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [6]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [8]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [9]:
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=5)


In [10]:
for index, site in launch_sites_df.iterrows():
    coord = [site['Lat'], site['Long']]

In [11]:
folium.Circle(
        coord,
        radius=1000,
        color='#007849',
        fill=True,
        fill_opacity=0.6,
        popup=site['Launch Site']
    ).add_to(site_map)

In [12]:
folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#007849;"><b>{site["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

In [13]:
site_map

In [16]:
site_map = folium.Map(location=[launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()], zoom_start=5)

# Add a circle and a label marker for each site
for index, site in launch_sites_df.iterrows():
    coord = [site['Lat'], site['Long']]
    
    # Circle around the launch site
    folium.Circle(
        coord,
        radius=1000,
        color='#007849',
        fill=True,
        fill_opacity=0.6,
        popup=site['Launch Site']
    ).add_to(site_map)

    # Label marker with site name
    folium.Marker(
        coord,
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#007849;"><b>{site["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Important: Display the map
site_map

<class 'ImportError'>: cannot import name 'document' from 'js' (unknown location)

In [17]:
site_success = spacex_df.groupby('Launch Site')['class'].agg(['count', 'sum']).reset_index()
site_success['success_rate'] = site_success['sum'] / site_success['count']

# Merge to get latitude and longitude for each site
launch_locations = spacex_df.groupby('Launch Site').agg({'Lat': 'first', 'Long': 'first'}).reset_index()
site_success = site_success.merge(launch_locations, on='Launch Site')

In [21]:
import folium
from folium.plugins import MarkerCluster

# Create map centered around average coordinates
site_map = folium.Map(location=[spacex_df['Lat'].mean(), spacex_df['Long'].mean()], zoom_start=4)

# Add clustered markers for each launch site
marker_cluster = MarkerCluster().add_to(site_map)

for index, row in site_success.iterrows():
    site = row['Launch Site']
    lat = row['Lat']
    lon = row['Long']
    rate = row['success_rate']
    total = row['count']
    success = row['sum']
    
    popup_text = (f"<b>Launch Site:</b> {site}<br>"
                  f"<b>Total Launches:</b> {total}<br>"
                  f"<b>Successful Launches:</b> {success}<br>"
                  f"<b>Success Rate:</b> {rate:.2%}")
    
    folium.CircleMarker(
        location=[lat, lon],
        radius=10,
        color='green' if rate >= 0.8 else 'orange' if rate >= 0.5 else 'red',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(popup_text, max_width=300)
    ).add_to(marker_cluster)

# Display the map
site_map.save("launch_success_map.html")

In [22]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [23]:
marker_cluster = MarkerCluster()

In [25]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map.save("launch_success_map1.html")

In [26]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance